In [1]:
import re
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup as bs
import extruct
from bs4 import BeautifulSoup
import time

In [2]:
url="https://www.lyrics.com/lyric/24488857/Pink+Floyd/Wish+You+Were+Here"
requests.get(url).content

b'\n<!doctype html>\n<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]--><head>\n<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<title>Pink Floyd - Wish You Were Here Lyrics | Lyrics.com</title>\n<meta name="description" content="Wish You Were Here Lyrics by Pink Floyd from the A Foot in the Door: The Best of Pink Floyd album - including song video, artist biography, translations and more: So, so you think you can tell\r\nHeaven from hell\r\nBlue skies from pain\r\nCan you tell a green field\r\nFrom a cold steel ra&hellip;">\n<meta name="keywords" content="Wish You Were Here lyrics, lyrics for Wish You Were Here, Wish You Were Here song, Wish You Were Here words, lyrics from A Foot in the Door: The Be

In [3]:
class Crawler:

    def __init__(self, urls=[]):
        self.visited_urls = []
        self.urls_to_visit = urls
        self.visited_song=[]
        self.data=""
        self.pattren=re.compile("https://www.lyrics.com/.*")

    def download_url(self, url):
        
        if self.pattren.match(url) and url not in self.visited_urls:      
            return requests.get(url).text
        return ""

    def get_linked_urls(self, url, html):
        soup = BeautifulSoup(html, 'html.parser')
        for link in soup.find_all('a'):
            path = link.get('href')
            if path and path.startswith('/'):
                path = urljoin(url, path)
            yield path

    def add_url_to_visit(self, url):
        if url==None:
            return
        pattren1=re.compile("/.*")
       
        if pattren1.match(url):
           
            url="https://www.lyrics.com"+url
        if url not in self.visited_urls and url not in self.urls_to_visit and url!=None and self.pattren.match(url):
            pattren1=re.compile("https://www.lyrics.com/lyric/.*")
            if pattren1.match(url):
                self.urls_to_visit.insert(0,url)
            else:    
                self.urls_to_visit.append(url)

    def crawl(self, url):
        html = self.download_url(url)
        title=""
        artist=""
        soup=bs(html,"html.parser")
        pt=re.compile(r'^(([0-9]|!|#|&|[a-zA-Z]| |-|,|\"|\.|\'|\(|\)))(([0-9]|[|]|\\|[a-zA-Z]| |-|,|\"|\.|\'|\(|\)))*')
        lyrics=soup.find("pre",attrs={"class":"lyric-body"})
        if lyrics!=None:
            
            if len(lyrics.get_text().split('\n'))<=1:
              
                lyrics=None
        if len(re.findall(r'<h1 id="lyric-title-text" class="lyric-title">(.*?)</h1>' ,html))>0  and len(re.findall(r'<h3 class="lyric-artist"><a href=".*">(.*?)</a></h3>' ,html))>0:
                title=re.findall(r'<h1 id="lyric-title-text" class="lyric-title">(.*?)</h1>' ,html)[0]
                artist=re.findall(r'<h3 class="lyric-artist"><a href=".*">(.*?)</a></h3>' ,html)[0]
        if title!="" and artist!=""and pt.fullmatch(title) and lyrics!=None and (artist+": "+title) not in self.visited_song:
            self.visited_song.append(artist+": "+title)
            self.data+=html
          
        for url in self.get_linked_urls(url, html):
              
                self.add_url_to_visit(url)

    def run(self,depth=20):
        while len(self.urls_to_visit)>0 and len(self.visited_song)<=depth:
            url = self.urls_to_visit.pop(0)
            
            try:
                print(len(self.urls_to_visit),len(self.visited_song),str(time.localtime().tm_hour)+" "+str(time.localtime().tm_min))
                self.crawl(url)
            except Exception:
                print(Exception)
            finally:
                if not(url in self.visited_urls):
                    self.visited_urls.append(url)
    def parse_item(self):
        t=self.data.split("</html>")
        u=[]
        h=dict()
        for i in range(len(t)):
            t[i]+="</html>"
            if len(re.findall(r'<h1 id="lyric-title-text" class="lyric-title">(.*?)</h1>' ,t[i]))>0 and len(re.findall(r'<h3 class="lyric-artist"><a href=".*">(.*?)</a></h3>' ,t[i]))>0:
                title=re.findall(r'<h1 id="lyric-title-text" class="lyric-title">(.*?)</h1>' ,t[i])[0]
                artist=re.findall(r'<h3 class="lyric-artist"><a href=".*">(.*?)</a></h3>' ,t[i])[0]
                soup=bs(t[i],"html.parser")
                lyrics=soup.find("pre",attrs={"class":"lyric-body"})
                h["lyrics"]=lyrics.get_text()
                h["title"]=title
                h["artist"]=artist
                u.append(h)
                h=dict()
        return u    

In [4]:
temp=Crawler()

In [5]:
temp.crawl(url)

In [6]:
temp.run(100)

63 1 22 18
69 2 22 19
69 2 22 19
68 2 22 19
67 2 22 19
71 3 22 19
71 3 22 19
70 3 22 19
69 3 22 19
73 4 22 19
73 4 22 19
72 4 22 19
71 4 22 19
74 5 22 19
74 5 22 19
73 5 22 19
72 5 22 19
78 6 22 19
78 6 22 19
77 6 22 19
76 6 22 19
81 7 22 19
81 7 22 19
80 7 22 19
79 7 22 19
83 8 22 19
83 8 22 19
82 8 22 19
81 8 22 19
85 9 22 19
85 9 22 19
84 9 22 19
83 9 22 19
92 9 22 19
97 10 22 19
97 10 22 19
96 10 22 19
96 10 22 19
95 10 22 19
100 11 22 19
100 11 22 19
99 11 22 19
98 11 22 19
104 12 22 19
104 12 22 19
103 12 22 19
103 12 22 19
102 12 22 19
134 13 22 19
138 14 22 19
138 14 22 19
137 14 22 19
136 14 22 19
141 15 22 19
141 15 22 19
140 15 22 19
139 15 22 19
143 15 22 19
143 15 22 19
142 15 22 19
141 15 22 19
147 16 22 19
147 16 22 20
146 16 22 20
145 16 22 20
150 17 22 20
150 17 22 20
149 17 22 20
148 17 22 20
153 18 22 20
153 18 22 20
152 18 22 20
151 18 22 20
156 19 22 20
156 19 22 20
155 19 22 20
154 19 22 20
162 20 22 20
162 20 22 20
161 20 22 20
160 20 22 20
166 21 22 20
166 21 22

temp.data

In [7]:
d=temp.parse_item()

In [8]:
len(d)

101

In [9]:
import json

In [10]:
final=json.dumps(d)

In [11]:
with open("lyrics.json", "w") as final:
    json.dump(d, final)

In [12]:
temp.visited_song

['Pink Floyd: Wish You Were Here',
 'Pink Floyd: Eclipse',
 'Pink Floyd: Brain Damage',
 'Pink Floyd: Shine on You Crazy Diamond, Pts. 1-5',
 'Pink Floyd: The Fletcher Memorial Home',
 'Pink Floyd: Learning to Fly',
 'Pink Floyd: High Hopes',
 'Pink Floyd: Comfortably Numb',
 'Pink Floyd: Money',
 'Nio Garcia: AM',
 'Eladio Carrion: Sauce Boy Freestyle 4',
 'Zion & Lennox: Loco',
 "The Beatles: Maxwell's Silver Hammer",
 'The Beatles: The Long One',
 'The Beatles: Come and Get It',
 'The Beatles: The End',
 'The Beatles: She Came in Through the Bathroom Window',
 'The Beatles: Polythene Pam',
 'The Beatles: Mean Mr. Mustard',
 'The Beatles: Because',
 'The Beatles: Something',
 'The Beatles: Come Together',
 'The Beatles: Old Brown Shoe',
 'The Beatles: The Ballad of John and Yoko',
 'The Beatles: Goodbye',
 'The Beatles: Her Majesty',
 'The Beatles: You Never Give Me Your Money',
 'The Beatles: Here Comes the Sun',
 "The Beatles: I Want You (She's So Heavy)",
 "The Beatles: Octopus's 

In [13]:
soup=bs("jjj","html.parser")
lyrics=soup.find("pre",attrs={"class":"lyric-body"})

In [14]:
lyrics==None

True

In [15]:
final=json.dumps(d)

In [16]:
final

'[{"lyrics": "So, so you think you can tell\\r\\nHeaven from hell\\r\\nBlue skies from pain\\r\\nCan you tell a green field\\r\\nFrom a cold steel rail?\\r\\nA smile from a veil?\\r\\nDo you think you can tell?\\r\\n\\r\\nDid they get you to trade\\r\\nYour heroes for ghosts?\\r\\nHot ashes for trees?\\r\\nHot air for a cool breeze?\\r\\nCold comfort for change?\\r\\nDid you exchange\\r\\nA walk on part in the war\\r\\nFor a lead role in a cage?\\r\\n\\r\\nHow I wish, how I wish you were here\\r\\nWe\'re just two lost souls\\r\\nSwimming in a fish bowl\\r\\nYear after year\\r\\nRunning over the same old ground\\r\\nAnd how we found\\r\\nThe same old fears\\r\\nWish you were here", "title": "Wish You Were Here", "artist": "Pink Floyd"}, {"lyrics": "All that you touch \\r\\nAll that you see \\r\\nAll that you taste \\r\\nAll you feel \\r\\nAll that you love \\r\\nAll that you hate \\r\\nAll you distrust \\r\\nAll you save \\r\\nAll that you give \\r\\nAll that you deal \\r\\nAll that you

In [17]:
t=temp.data.split("</html>")
u=[]
h=dict()
for i in range(len(t)):
            t[i]+="</html>"
            if len(re.findall(r'<h1 id="lyric-title-text" class="lyric-title">(.*?)</h1>' ,t[i]))>0 and len(re.findall(r'<h3 class="lyric-artist"><a href=".*">(.*?)</a></h3>' ,t[i]))>0:
                
                title=re.findall(r'<h1 id="lyric-title-text" class="lyric-title">(.*?)</h1>' ,t[i])[0]
                artist=re.findall(r'<h3 class="lyric-artist"><a href=".*">(.*?)</a></h3>' ,t[i])[0]
                soup=bs(t[i],"html.parser")
                lyrics=soup.find("pre",attrs={"class":"lyric-body"})
                h["lyrics"]=lyrics.get_text()
                h["title"]=title
                h["artist"]=artist
                u.append(h)
                h=dict()

In [18]:
u

[{'lyrics': "So, so you think you can tell\r\nHeaven from hell\r\nBlue skies from pain\r\nCan you tell a green field\r\nFrom a cold steel rail?\r\nA smile from a veil?\r\nDo you think you can tell?\r\n\r\nDid they get you to trade\r\nYour heroes for ghosts?\r\nHot ashes for trees?\r\nHot air for a cool breeze?\r\nCold comfort for change?\r\nDid you exchange\r\nA walk on part in the war\r\nFor a lead role in a cage?\r\n\r\nHow I wish, how I wish you were here\r\nWe're just two lost souls\r\nSwimming in a fish bowl\r\nYear after year\r\nRunning over the same old ground\r\nAnd how we found\r\nThe same old fears\r\nWish you were here",
  'title': 'Wish You Were Here',
  'artist': 'Pink Floyd'},
 {'lyrics': "All that you touch \r\nAll that you see \r\nAll that you taste \r\nAll you feel \r\nAll that you love \r\nAll that you hate \r\nAll you distrust \r\nAll you save \r\nAll that you give \r\nAll that you deal \r\nAll that you buy  \r\nBeg, borrow or steal \r\nAll you create \r\nAll you des

In [19]:
with open("lyrics.json", "w") as final:
    json.dump(u, final)

In [20]:
pt=re.compile(r'^(([0-9]|!|#|&|[a-zA-Z]| |-|,|\"|\.|\'|\(|\)))(([0-9]|[|]|\\|[a-zA-Z]| |-|,|\"|\.|\'|\(|\)))*')
cleared_result=[]
for i in u:
    if pt.fullmatch(i["title"]):
        cleared_result.append(i["artist"]+": "+i["title"])

In [21]:
for i in temp.visited_song:
    found=False
    for j in cleared_result:
        if i==j:
            found=True
            print(i,j)
    if not found:        
        print("not found",i)        

Pink Floyd: Wish You Were Here Pink Floyd: Wish You Were Here
Pink Floyd: Eclipse Pink Floyd: Eclipse
Pink Floyd: Brain Damage Pink Floyd: Brain Damage
Pink Floyd: Shine on You Crazy Diamond, Pts. 1-5 Pink Floyd: Shine on You Crazy Diamond, Pts. 1-5
Pink Floyd: The Fletcher Memorial Home Pink Floyd: The Fletcher Memorial Home
Pink Floyd: Learning to Fly Pink Floyd: Learning to Fly
Pink Floyd: High Hopes Pink Floyd: High Hopes
Pink Floyd: Comfortably Numb Pink Floyd: Comfortably Numb
Pink Floyd: Money Pink Floyd: Money
Nio Garcia: AM Nio Garcia: AM
Eladio Carrion: Sauce Boy Freestyle 4 Eladio Carrion: Sauce Boy Freestyle 4
Zion & Lennox: Loco Zion & Lennox: Loco
The Beatles: Maxwell's Silver Hammer The Beatles: Maxwell's Silver Hammer
The Beatles: The Long One The Beatles: The Long One
The Beatles: Come and Get It The Beatles: Come and Get It
The Beatles: The End The Beatles: The End
The Beatles: She Came in Through the Bathroom Window The Beatles: She Came in Through the Bathroom Windo

In [22]:
print(pt.findall("I Believe In You"))

[('I', 'I', 'u', 'u')]


In [23]:
pattren=re.compile("(https://www.lyrics.com/.*)|(/lyric/.*)")

In [24]:
pattren.match("/lyric/y/kk")

<re.Match object; span=(0, 11), match='/lyric/y/kk'>

In [25]:
str(time.localtime().tm_hour)+" "+str(time.localtime().tm_min)

'22 28'